# Goal : Generate the final Dataframe with the non redundant protein sequences, with all the relevant information
***
## I. DepoScope method (Anubis return)
## II. HMM profile DB (PPT)
## III. Blastp Bea DB (Minibatch)
***

> Anubis return

In [3]:
import os 
import pandas as pd
from tqdm import tqdm 
import warnings
warnings.filterwarnings('ignore')

# local :
path_project = "/media/concha-eloko/Linux/PPT_clean"

df_anubis_return = pd.read_csv(f"{path_project}/embeddings/anubis_return.esm2.embedding.csv" , sep = ",", header = None)


In [4]:
df_anubis_return

,0,1,2,3,4,5,6,7,8,9,...,1272,1273,1274,1275,1276,1277,1278,1279,1280,1281
0,anubis_return__3527,0.038820,0.060806,0.009003,-0.000551,-0.079944,0.004779,0.002953,-0.054475,0.080076,...,-0.008636,0.086445,0.008170,0.021060,-0.001021,0.039339,-0.097045,0.055026,0.063590,NaN
1,anubis_return__1471,0.047835,0.013409,0.007318,0.016219,-0.089094,0.013079,-0.027569,-0.096018,0.073152,...,0.060111,0.124358,0.035561,-0.010983,0.054189,0.048831,-0.015946,0.041427,0.025712,NaN
2,anubis_return__324,0.052343,-0.028331,-0.027254,0.067171,-0.141176,0.031074,0.107378,-0.202795,0.046812,...,-0.033496,0.005376,0.026139,-0.050430,-0.018309,0.060166,-0.132078,-0.180234,0.159205,NaN
3,anubis_return__1447,-0.000731,0.057064,0.009525,-0.023543,-0.079982,0.023000,0.038574,-0.055469,0.118188,...,0.000178,0.133065,-0.028866,0.062627,0.031955,0.009178,-0.104672,0.059477,0.135439,NaN
4,anubis_return__2709,0.016032,0.054470,-0.004944,-0.009878,-0.096860,0.033161,0.030150,-0.091246,0.038327,...,-0.022917,0.056102,0.018805,0.005814,0.041827,0.040712,-0.089219,0.028831,0.055344,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4303,anubis_return__3502,-0.012740,0.058042,-0.025648,-0.032004,-0.100340,0.021771,0.039250,-0.062221,0.077716,...,-0.007077,0.110891,-0.003376,0.046269,0.068144,0.046381,-0.111445,0.059284,0.064103,NaN
4304,anubis_return__2113,-0.010991,0.031142,-0.000453,-0.015756,-0.087222,0.026853,0.018421,-0.114121,0.037754,...,-0.014907,0.104043,0.006727,0.034564,0.056344,0.038118,-0.064529,0.033074,0.046867,NaN
4305,anubis_return__209,0.008060,0.062799,-0.019679,0.000484,-0.097984,0.009262,0.015033,-0.034346,0.061062,...,-0.001716,0.110625,0.008589,0.064329,0.051080,-0.028424,-0.137705,0.085697,0.056727,NaN
4306,anubis_return__2441,0.031140,0.018862,-0.010451,0.021861,-0.088719,0.043387,-0.034558,0.118990,0.075993,...,-0.008831,0.022576,0.068691,0.035253,0.162201,0.019593,-0.031738,-0.032016,-0.014962,NaN


In [ ]:
import os 
import pandas as pd
from tqdm import tqdm 
import warnings
warnings.filterwarnings('ignore')

# local :
path_project = "/media/concha-eloko/Linux/PPT_clean"
path_similarity = path_project

#esm2_depo = pd.read_csv(f"{path_project}/Dpo.0805.embeddings.ultimate.csv" , header = None, sep = "," , index_col = 0)
df_info = pd.read_csv(f"{path_project}/DF_Dpo.final.1005.tsv" , sep = "\t", header =0)
family_df = pd.read_csv(f"{path_similarity}/clusters_99_80.extra_clean.2004.v2.tsv", sep = "\t", header = 0)
dpo_final_info = pd.read_csv(f"{path_project}/Dpo_domains.0805.final.ultimate.tsv" , sep = "\t", names = ["0","Dpo","Fold","Prob","Domain_Boundaries","aa_domain","aa_full"])
dpo_final_info = dpo_final_info.drop(["0"] , axis = 1)

In [ ]:
df_dpo_info = df_info[df_info["index"].isin(esm2_depo.index)]
df_dpo_info_ultimate = df_dpo_info.drop_duplicates(subset = ["Infected_ancestor","index"], keep = "first")

df_dpo_info_ultimate[df_dpo_info_ultimate["Infected_ancestor"] == "n47954795"]